# 5.4 **Evaluate and Tune** Neural Networks - Predict Student Departure

## Model Cycle: The 5 Key Steps

### 1. Build the Model : Create the Neural Network architecture with Keras.  
### 2. Train the Model : Fit the model on the training data.  
### 3. Generate Predictions : Use the trained model to make predictions.  
### **4. Evaluate the Model : Assess performance using evaluation metrics.**  
### **5. Improve the Model : Tune hyperparameters for optimal performance.**

## Introduction

In this final notebook of Module 5, we comprehensively evaluate our trained neural networks and explore techniques to improve their performance.

We will:
1. Evaluate models on the held-out test set
2. Compare neural networks with tree-based models (Random Forest)
3. Apply **dropout regularization** to prevent overfitting
4. Tune network architecture and hyperparameters
5. Provide recommendations for when to use neural networks

### Learning Objectives

By the end of this notebook, you will be able to:

1. Evaluate neural network performance using multiple metrics
2. Compare neural networks with tree-based models for tabular data
3. Implement dropout regularization in Keras
4. Tune neural network architecture (layers, neurons)
5. Make informed decisions about when to use neural networks

## 1. Load Dependencies and Models

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Core libraries
import pandas as pd
import numpy as np
import pickle
import time

# Visualization
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# Scikit-learn metrics
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, confusion_matrix, classification_report
)

# TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# Display settings
pd.options.display.max_columns = None

# Set random seeds
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")

In [ ]:
# Set up file paths
root_filepath = '/content/drive/MyDrive/projects/Applied-Data-Analytics-For-Higher-Education-Course-2/'
data_filepath = f'{root_filepath}data/'
course3_filepath = f'{root_filepath}course_3/'
module5_filepath = f'{course3_filepath}module_5/'
models_path = f'{module5_filepath}models/'

In [ ]:
# Load the trained models
model_simple = keras.models.load_model(f'{models_path}simple_nn_trained.keras')
model_deep = keras.models.load_model(f'{models_path}deep_nn_trained.keras')
model_wide = keras.models.load_model(f'{models_path}wide_nn_trained.keras')

models = {
    'Simple NN': model_simple,
    'Deep NN': model_deep,
    'Wide NN': model_wide
}

print("Loaded trained models:")
for name, model in models.items():
    print(f"  {name}: {model.count_params()} parameters")

In [ ]:
# Load preprocessor and feature info
preprocessor = pickle.load(open(f'{models_path}preprocessor.pkl', 'rb'))
feature_info = pickle.load(open(f'{models_path}feature_info.pkl', 'rb'))

all_features = feature_info['all_features']
input_dim = feature_info['input_dim']

print(f"Features: {all_features}")
print(f"Input dimension: {input_dim}")

In [ ]:
# Load test data
df_testing = pd.read_csv(f'{data_filepath}testing.csv')

print(f"Test data shape: {df_testing.shape}")
print(f"\nTarget distribution:")
print(df_testing['SEM_3_STATUS'].value_counts())

In [ ]:
# Prepare test data
X_test = df_testing[all_features]
y_test = df_testing['SEM_3_STATUS'].values

# Transform features
X_test_processed = preprocessor.transform(X_test)

print(f"Test set: {X_test_processed.shape[0]} samples, {X_test_processed.shape[1]} features")

In [ ]:
# Also load training data for later use
data_splits = pickle.load(open(f'{models_path}data_splits.pkl', 'rb'))
X_train = data_splits['X_train']
X_val = data_splits['X_val']
y_train = data_splits['y_train']
y_val = data_splits['y_val']

print(f"Training set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples")
print(f"Test set: {X_test_processed.shape[0]} samples")

## 2. Evaluate on Test Data

The test set provides an unbiased estimate of model performance on new, unseen data.

### 2.1 Generate Test Predictions

In [ ]:
# Generate predictions on test data
test_predictions = {}
test_probabilities = {}

for name, model in models.items():
    # Get probability predictions
    probs = model.predict(X_test_processed, verbose=0)
    test_probabilities[name] = probs.flatten()
    
    # Convert to class predictions
    test_predictions[name] = (probs > 0.5).astype(int).flatten()
    
print("Test predictions generated for all models.")

In [ ]:
# Calculate test metrics
test_metrics = []

for name in models.keys():
    y_pred = test_predictions[name]
    y_prob = test_probabilities[name]
    
    test_metrics.append({
        'Model': name,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred),
        'ROC-AUC': roc_auc_score(y_test, y_prob)
    })

test_metrics_df = pd.DataFrame(test_metrics)
print("Test Set Performance:")
test_metrics_df

### 2.2 Confusion Matrices

In [ ]:
# Plot confusion matrices
fig = make_subplots(rows=1, cols=3, subplot_titles=list(models.keys()))

for idx, (name, y_pred) in enumerate(test_predictions.items(), 1):
    cm = confusion_matrix(y_test, y_pred)
    
    fig.add_trace(
        go.Heatmap(
            z=cm,
            x=['Pred: Retained', 'Pred: Departed'],
            y=['True: Retained', 'True: Departed'],
            colorscale='Blues',
            showscale=False,
            text=cm,
            texttemplate='%{text}',
            textfont=dict(size=14)
        ),
        row=1, col=idx
    )

fig.update_layout(
    title='Confusion Matrices (Test Set)',
    height=400
)

fig.show()

### 2.3 Classification Reports

In [ ]:
# Print classification reports
for name in models.keys():
    print(f"\n{'='*60}")
    print(f"Classification Report: {name}")
    print('='*60)
    print(classification_report(y_test, test_predictions[name], 
                               target_names=['Retained', 'Departed']))

### 2.4 ROC Curves and AUC

In [ ]:
# Plot ROC curves
fig = go.Figure()

colors = {'Simple NN': 'blue', 'Deep NN': 'green', 'Wide NN': 'orange'}

for name in models.keys():
    fpr, tpr, _ = roc_curve(y_test, test_probabilities[name])
    auc = roc_auc_score(y_test, test_probabilities[name])
    
    fig.add_trace(go.Scatter(
        x=fpr, y=tpr,
        mode='lines',
        name=f'{name} (AUC={auc:.4f})',
        line=dict(color=colors[name], width=2)
    ))

# Add diagonal reference line
fig.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1],
    mode='lines',
    name='Random (AUC=0.5)',
    line=dict(color='gray', width=1, dash='dash')
))

fig.update_layout(
    title='ROC Curves - Neural Networks (Test Set)',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    xaxis=dict(range=[0, 1]),
    yaxis=dict(range=[0, 1]),
    height=500
)

fig.show()

## 3. Compare with Tree-Based Models

### 3.1 Load Random Forest Model

Let's compare our neural networks with the Random Forest model from Module 3.

In [ ]:
# Load Random Forest model from Module 3
try:
    rf_model = pickle.load(open(f'{course3_filepath}models/rf_best_trained_model.pkl', 'rb'))
    print("Random Forest model loaded successfully.")
    print(f"Number of trees: {rf_model.named_steps['classifier'].n_estimators}")
except FileNotFoundError:
    print("Random Forest model not found. Training a new one...")
    
    # If not found, we'll create and train a Random Forest
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.pipeline import Pipeline
    from sklearn.compose import ColumnTransformer
    from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
    
    # Load raw training data
    df_training = pd.read_csv(f'{data_filepath}training.csv')
    
    # Define columns
    minmax_columns = ['HS_GPA', 'GPA_1', 'GPA_2', 'DFW_RATE_1', 'DFW_RATE_2']
    standard_columns = ['UNITS_ATTEMPTED_1', 'UNITS_ATTEMPTED_2']
    categorical_columns = ['GENDER', 'RACE_ETHNICITY', 'FIRST_GEN_STATUS']
    
    # Build preprocessor
    rf_preprocessor = ColumnTransformer([
        ('minmax', MinMaxScaler(), minmax_columns),
        ('standard', StandardScaler(), standard_columns),
        ('onehot', OneHotEncoder(handle_unknown='ignore', 
                                  drop=['Female', 'Other', 'Unknown'], 
                                  sparse_output=False), categorical_columns)
    ])
    
    # Build pipeline
    rf_model = Pipeline([
        ('preprocessing', rf_preprocessor),
        ('classifier', RandomForestClassifier(
            n_estimators=100,
            max_depth=None,
            class_weight='balanced',
            random_state=42,
            n_jobs=-1
        ))
    ])
    
    # Train
    rf_model.fit(df_training[all_features], df_training['SEM_3_STATUS'])
    print("Random Forest trained successfully.")

In [ ]:
# Generate Random Forest predictions on test data
rf_predictions = rf_model.predict(df_testing[all_features])
rf_probabilities = rf_model.predict_proba(df_testing[all_features])[:, 1]

print("Random Forest predictions generated.")

### 3.2 Performance Comparison

In [ ]:
# Add Random Forest to comparison
all_models = {
    'Simple NN': (test_predictions['Simple NN'], test_probabilities['Simple NN']),
    'Deep NN': (test_predictions['Deep NN'], test_probabilities['Deep NN']),
    'Wide NN': (test_predictions['Wide NN'], test_probabilities['Wide NN']),
    'Random Forest': (rf_predictions, rf_probabilities)
}

comparison_results = []

for name, (y_pred, y_prob) in all_models.items():
    comparison_results.append({
        'Model': name,
        'Type': 'Neural Network' if 'NN' in name else 'Tree-Based',
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision': precision_score(y_test, y_pred),
        'Recall': recall_score(y_test, y_pred),
        'F1-Score': f1_score(y_test, y_pred),
        'ROC-AUC': roc_auc_score(y_test, y_prob)
    })

comparison_df = pd.DataFrame(comparison_results)
print("Model Comparison (Test Set):")
comparison_df

In [ ]:
# Visualize comparison
fig = go.Figure()

metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']
colors_all = ['blue', 'green', 'orange', 'purple']

for i, row in comparison_df.iterrows():
    values = [row[m] for m in metrics]
    fig.add_trace(go.Bar(
        name=row['Model'],
        x=metrics,
        y=values,
        marker_color=colors_all[i]
    ))

fig.update_layout(
    title='Model Performance Comparison: Neural Networks vs. Random Forest',
    xaxis_title='Metric',
    yaxis_title='Score',
    barmode='group',
    yaxis=dict(range=[0, 1]),
    height=500
)

fig.show()

In [ ]:
# Compare ROC curves including Random Forest
fig = go.Figure()

all_colors = {'Simple NN': 'blue', 'Deep NN': 'green', 'Wide NN': 'orange', 'Random Forest': 'purple'}

for name, (y_pred, y_prob) in all_models.items():
    fpr, tpr, _ = roc_curve(y_test, y_prob)
    auc = roc_auc_score(y_test, y_prob)
    
    fig.add_trace(go.Scatter(
        x=fpr, y=tpr,
        mode='lines',
        name=f'{name} (AUC={auc:.4f})',
        line=dict(color=all_colors[name], width=2)
    ))

# Add diagonal
fig.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1],
    mode='lines',
    name='Random',
    line=dict(color='gray', width=1, dash='dash')
))

fig.update_layout(
    title='ROC Curves: Neural Networks vs. Random Forest',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    height=500
)

fig.show()

### 3.3 When to Use Each Model

In [ ]:
# Create recommendation table
recommendations = pd.DataFrame({
    'Factor': [
        'Dataset Size',
        'Feature Types',
        'Training Time',
        'Interpretability',
        'Missing Data',
        'Feature Engineering',
        'Non-linear Patterns',
        'Scalability'
    ],
    'Random Forest': [
        'Works well with smaller datasets (100s-1000s)',
        'Handles mixed types natively',
        'Generally faster',
        'Feature importance available',
        'Handles naturally',
        'Less needed',
        'Good',
        'Memory intensive for large forests'
    ],
    'Neural Network': [
        'Needs more data (1000s+)',
        'Requires encoding/scaling',
        'Slower (requires tuning)',
        'Black box (needs SHAP/LIME)',
        'Requires imputation',
        'Can learn features automatically',
        'Excellent',
        'GPU acceleration available'
    ]
})

print("When to Use Each Model:")
recommendations

In [ ]:
# Summary for our dataset
print("Recommendation for Student Departure Prediction:")
print("="*60)
print("\nOur dataset characteristics:")
print(f"  - Size: ~{len(y_train)} training samples (moderate)")
print(f"  - Features: {input_dim} (small)")
print("  - Feature types: Mixed (numeric + categorical)")
print("  - Problem: Binary classification")
print("\nBased on these characteristics:")
print("  - Both Random Forest and Neural Networks can work well")
print("  - Random Forest may be preferred for:")
print("    * Faster training and deployment")
print("    * Better interpretability for stakeholders")
print("    * Robust performance without extensive tuning")
print("  - Neural Networks may be preferred for:")
print("    * When more data becomes available")
print("    * When deployed as part of larger deep learning system")
print("    * When capture complex interactions is critical")

## 4. Dropout Regularization

### 4.1 What is Dropout?

**Dropout** is a regularization technique that randomly "drops" neurons during training:

- Each neuron has a probability `p` of being temporarily removed
- Forces the network to not rely on any single neuron
- Creates an implicit ensemble of many sub-networks
- At test time, all neurons are used (with scaled weights)

**Typical dropout rates:**
- Input layer: 0.1-0.2 (lighter dropout)
- Hidden layers: 0.2-0.5 (heavier dropout)

In [ ]:
# Visualize dropout concept
def visualize_dropout():
    fig = make_subplots(rows=1, cols=2, subplot_titles=(
        'Without Dropout (All Neurons Active)',
        'With Dropout (Some Neurons Dropped)'
    ))
    
    # Network without dropout
    np.random.seed(42)
    
    # Draw neurons
    layer_positions = [0, 1.5, 3]
    layer_sizes = [4, 6, 2]
    
    for col in [1, 2]:
        dropped = np.zeros((3, 6), dtype=bool)
        if col == 2:  # With dropout
            dropped[1] = np.random.random(6) < 0.3  # 30% dropout
        
        for layer_idx, (x, n_neurons) in enumerate(zip(layer_positions, layer_sizes)):
            for neuron_idx in range(n_neurons):
                y = (6 - n_neurons) / 2 + neuron_idx
                is_dropped = dropped[layer_idx, neuron_idx] if neuron_idx < 6 else False
                
                color = 'lightgray' if is_dropped else (['lightblue', 'lightgreen', 'lightyellow'][layer_idx])
                
                fig.add_trace(go.Scatter(
                    x=[x], y=[y],
                    mode='markers',
                    marker=dict(size=25, color=color, line=dict(width=2, color='gray')),
                    showlegend=False
                ), row=1, col=col)
                
                # Draw connections
                if layer_idx < 2:
                    next_size = layer_sizes[layer_idx + 1]
                    for next_idx in range(next_size):
                        next_y = (6 - next_size) / 2 + next_idx
                        
                        # Check if either neuron is dropped
                        next_dropped = dropped[layer_idx + 1, next_idx] if next_idx < 6 else False
                        line_color = 'lightgray' if (is_dropped or next_dropped) else 'gray'
                        line_width = 0.5 if (is_dropped or next_dropped) else 1
                        
                        fig.add_trace(go.Scatter(
                            x=[x, layer_positions[layer_idx + 1]],
                            y=[y, next_y],
                            mode='lines',
                            line=dict(color=line_color, width=line_width),
                            showlegend=False
                        ), row=1, col=col)
    
    fig.update_layout(
        title='Dropout Regularization: Randomly Dropping Neurons During Training',
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        height=400
    )
    
    return fig

fig = visualize_dropout()
fig.show()

### 4.2 Build Model with Dropout

In [ ]:
# Create model with dropout
def create_dropout_model(input_dim, dropout_rate=0.3):
    """
    Create a neural network with dropout regularization.
    
    Parameters:
    -----------
    input_dim : int
        Number of input features
    dropout_rate : float
        Probability of dropping neurons (0-1)
    """
    model = Sequential([
        Input(shape=(input_dim,)),
        
        # Hidden layer 1 with dropout
        Dense(32, activation='relu'),
        Dropout(dropout_rate),
        
        # Hidden layer 2 with dropout
        Dense(16, activation='relu'),
        Dropout(dropout_rate),
        
        # Output layer (no dropout)
        Dense(1, activation='sigmoid')
    ], name='dropout_nn')
    
    return model

# Create and compile
model_dropout = create_dropout_model(input_dim, dropout_rate=0.3)
model_dropout.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

print("Model with Dropout:")
model_dropout.summary()

### 4.3 Train and Evaluate Dropout Model

In [ ]:
# Calculate class weights
n_class_0 = (y_train == 0).sum()
n_class_1 = (y_train == 1).sum()
total = len(y_train)

class_weights = {
    0: total / (2 * n_class_0),
    1: total / (2 * n_class_1)
}

# Training callbacks
callbacks_dropout = [
    EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001, verbose=1)
]

In [ ]:
# Train the dropout model
print("Training Neural Network with Dropout...")
print("="*50)

start_time = time.time()

history_dropout = model_dropout.fit(
    X_train, y_train,
    epochs=100,
    batch_size=32,
    validation_data=(X_val, y_val),
    class_weight=class_weights,
    callbacks=callbacks_dropout,
    verbose=1
)

training_time_dropout = time.time() - start_time
print(f"\nTraining completed in {training_time_dropout:.2f} seconds")

In [ ]:
# Plot dropout model training history
hist = history_dropout.history
epochs = range(1, len(hist['loss']) + 1)

fig = make_subplots(rows=1, cols=2, subplot_titles=('Loss', 'AUC'))

fig.add_trace(go.Scatter(x=list(epochs), y=hist['loss'], name='Train', line=dict(color='blue')), row=1, col=1)
fig.add_trace(go.Scatter(x=list(epochs), y=hist['val_loss'], name='Val', line=dict(color='red')), row=1, col=1)

fig.add_trace(go.Scatter(x=list(epochs), y=hist['auc'], name='Train', line=dict(color='blue'), showlegend=False), row=1, col=2)
fig.add_trace(go.Scatter(x=list(epochs), y=hist['val_auc'], name='Val', line=dict(color='red'), showlegend=False), row=1, col=2)

fig.update_xaxes(title='Epoch')
fig.update_layout(
    title='Dropout Neural Network Training History',
    height=400
)

fig.show()

In [ ]:
# Evaluate dropout model on test set
dropout_probs = model_dropout.predict(X_test_processed, verbose=0).flatten()
dropout_preds = (dropout_probs > 0.5).astype(int)

print("Dropout Neural Network Test Performance:")
print("="*50)
print(f"Accuracy:  {accuracy_score(y_test, dropout_preds):.4f}")
print(f"Precision: {precision_score(y_test, dropout_preds):.4f}")
print(f"Recall:    {recall_score(y_test, dropout_preds):.4f}")
print(f"F1-Score:  {f1_score(y_test, dropout_preds):.4f}")
print(f"ROC-AUC:   {roc_auc_score(y_test, dropout_probs):.4f}")

## 5. Architecture Tuning

### 5.1 Hyperparameter Search Space

Key hyperparameters for neural network architecture:

| Hyperparameter | Typical Range | Effect |
|:---------------|:--------------|:-------|
| Number of layers | 1-5 | Complexity of patterns learned |
| Neurons per layer | 8-256 | Model capacity |
| Dropout rate | 0.1-0.5 | Regularization strength |
| Learning rate | 0.0001-0.01 | Training speed/stability |
| Batch size | 16-256 | Training dynamics |

### 5.2 Manual Architecture Tuning

Let's try several different architectures and compare their performance.

In [ ]:
# Define architectures to test
architectures = {
    'Tiny (8)': [8],
    'Small (16)': [16],
    'Medium (32, 16)': [32, 16],
    'Large (64, 32, 16)': [64, 32, 16],
    'Deep (32, 32, 32, 16)': [32, 32, 32, 16],
    'Wide (128, 64)': [128, 64]
}

def create_model_from_architecture(input_dim, hidden_layers, dropout_rate=0.2):
    """Create a model from a list of hidden layer sizes."""
    layers = [Input(shape=(input_dim,))]
    
    for units in hidden_layers:
        layers.append(Dense(units, activation='relu'))
        layers.append(Dropout(dropout_rate))
    
    layers.append(Dense(1, activation='sigmoid'))
    
    model = Sequential(layers)
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )
    
    return model

print("Architectures to test:")
for name, layers in architectures.items():
    model = create_model_from_architecture(input_dim, layers)
    print(f"  {name}: {model.count_params()} parameters")

In [ ]:
# Train and evaluate each architecture
architecture_results = []

for name, hidden_layers in architectures.items():
    print(f"\nTraining {name}...")
    
    # Create model
    model = create_model_from_architecture(input_dim, hidden_layers)
    
    # Train with early stopping
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=0)
    ]
    
    start_time = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_data=(X_val, y_val),
        class_weight=class_weights,
        callbacks=callbacks,
        verbose=0
    )
    training_time = time.time() - start_time
    
    # Evaluate on test set
    probs = model.predict(X_test_processed, verbose=0).flatten()
    preds = (probs > 0.5).astype(int)
    
    architecture_results.append({
        'Architecture': name,
        'Hidden Layers': str(hidden_layers),
        'Parameters': model.count_params(),
        'Epochs': len(history.history['loss']),
        'Train Time (s)': round(training_time, 2),
        'Test Accuracy': round(accuracy_score(y_test, preds), 4),
        'Test AUC': round(roc_auc_score(y_test, probs), 4)
    })
    
    print(f"  Test AUC: {roc_auc_score(y_test, probs):.4f}")

print("\nArchitecture tuning complete!")

### 5.3 Results Analysis

In [ ]:
# Display results
arch_df = pd.DataFrame(architecture_results)
arch_df = arch_df.sort_values('Test AUC', ascending=False)

print("Architecture Tuning Results (sorted by Test AUC):")
arch_df

In [ ]:
# Visualize architecture comparison
fig = make_subplots(rows=1, cols=2, subplot_titles=(
    'Test AUC by Architecture',
    'Parameters vs. Performance'
))

# Bar chart of AUC
fig.add_trace(go.Bar(
    x=arch_df['Architecture'],
    y=arch_df['Test AUC'],
    marker_color='steelblue',
    text=arch_df['Test AUC'].round(4),
    textposition='outside'
), row=1, col=1)

# Scatter plot of parameters vs AUC
fig.add_trace(go.Scatter(
    x=arch_df['Parameters'],
    y=arch_df['Test AUC'],
    mode='markers+text',
    text=arch_df['Architecture'],
    textposition='top center',
    marker=dict(size=15, color='steelblue')
), row=1, col=2)

fig.update_xaxes(title='Architecture', row=1, col=1)
fig.update_xaxes(title='Number of Parameters', row=1, col=2)
fig.update_yaxes(title='Test AUC')

fig.update_layout(
    title='Neural Network Architecture Comparison',
    height=450,
    showlegend=False
)

fig.show()

In [ ]:
# Best architecture
best_arch = arch_df.iloc[0]
print(f"Best Architecture: {best_arch['Architecture']}")
print(f"  Hidden Layers: {best_arch['Hidden Layers']}")
print(f"  Parameters: {best_arch['Parameters']}")
print(f"  Test AUC: {best_arch['Test AUC']}")

## 6. Learning Rate Tuning

In [ ]:
# Test different learning rates with the best architecture
learning_rates = [0.01, 0.005, 0.001, 0.0005, 0.0001]

# Get best architecture
best_layers = eval(best_arch['Hidden Layers'])  # Convert string back to list

lr_results = []

for lr in learning_rates:
    print(f"\nTesting learning rate: {lr}")
    
    # Create model
    model = create_model_from_architecture(input_dim, best_layers, dropout_rate=0.2)
    model.compile(
        optimizer=Adam(learning_rate=lr),
        loss='binary_crossentropy',
        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
    )
    
    # Train
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=0)
    ]
    
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=32,
        validation_data=(X_val, y_val),
        class_weight=class_weights,
        callbacks=callbacks,
        verbose=0
    )
    
    # Evaluate
    probs = model.predict(X_test_processed, verbose=0).flatten()
    preds = (probs > 0.5).astype(int)
    
    lr_results.append({
        'Learning Rate': lr,
        'Epochs': len(history.history['loss']),
        'Test Accuracy': round(accuracy_score(y_test, preds), 4),
        'Test AUC': round(roc_auc_score(y_test, probs), 4)
    })
    
    print(f"  Test AUC: {roc_auc_score(y_test, probs):.4f}")

print("\nLearning rate tuning complete!")

In [ ]:
# Display learning rate results
lr_df = pd.DataFrame(lr_results)
lr_df = lr_df.sort_values('Test AUC', ascending=False)

print("Learning Rate Tuning Results:")
lr_df

In [ ]:
# Visualize learning rate results
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=lr_df['Learning Rate'],
    y=lr_df['Test AUC'],
    mode='lines+markers',
    marker=dict(size=12, color='steelblue'),
    line=dict(color='steelblue', width=2)
))

fig.update_layout(
    title='Test AUC vs. Learning Rate',
    xaxis_title='Learning Rate',
    yaxis_title='Test AUC',
    xaxis_type='log',  # Log scale for learning rate
    height=400
)

fig.show()

## 7. Final Model Comparison

In [ ]:
# Compile all results including tuned models
final_comparison = []

# Original models
for name, (y_pred, y_prob) in all_models.items():
    final_comparison.append({
        'Model': name,
        'Type': 'Neural Network' if 'NN' in name else 'Tree-Based',
        'Test AUC': round(roc_auc_score(y_test, y_prob), 4),
        'Test F1': round(f1_score(y_test, y_pred), 4)
    })

# Dropout model
final_comparison.append({
    'Model': 'Dropout NN',
    'Type': 'Neural Network',
    'Test AUC': round(roc_auc_score(y_test, dropout_probs), 4),
    'Test F1': round(f1_score(y_test, dropout_preds), 4)
})

# Best tuned architecture
final_comparison.append({
    'Model': f'Tuned NN ({best_arch["Architecture"]})',
    'Type': 'Neural Network (Tuned)',
    'Test AUC': best_arch['Test AUC'],
    'Test F1': round(f1_score(y_test, preds), 4)  # from last trained model
})

final_df = pd.DataFrame(final_comparison)
final_df = final_df.sort_values('Test AUC', ascending=False)

print("Final Model Comparison (sorted by Test AUC):")
final_df

In [ ]:
# Final visualization
fig = go.Figure()

# Color by type
colors_final = ['blue' if 'Neural' in t else 'green' for t in final_df['Type']]

fig.add_trace(go.Bar(
    x=final_df['Model'],
    y=final_df['Test AUC'],
    marker_color=colors_final,
    text=final_df['Test AUC'],
    textposition='outside'
))

fig.update_layout(
    title='Final Model Comparison: All Models (Test Set)',
    xaxis_title='Model',
    yaxis_title='Test AUC',
    yaxis=dict(range=[0.5, 1.0]),
    height=500
)

fig.add_annotation(
    x=0.5, y=0.95,
    xref='paper', yref='paper',
    text='Blue = Neural Network, Green = Tree-Based',
    showarrow=False,
    font=dict(size=12)
)

fig.show()

## 8. Best Practices and Recommendations

In [ ]:
# Print recommendations
print("Neural Network Best Practices for Student Departure Prediction")
print("="*70)

print("\n1. DATA PREPARATION")
print("   - Always scale numeric features (MinMax or Standard scaling)")
print("   - One-hot encode categorical variables")
print("   - Use class weights for imbalanced data")

print("\n2. ARCHITECTURE SELECTION")
print("   - Start simple (1-2 hidden layers)")
print("   - For tabular data: 16-64 neurons per layer is usually sufficient")
print("   - Use 'funnel' shape (wider at input, narrower toward output)")

print("\n3. REGULARIZATION")
print("   - Use dropout (0.2-0.3 is a good starting point)")
print("   - Early stopping prevents overfitting automatically")
print("   - Consider L2 regularization on Dense layers")

print("\n4. TRAINING")
print("   - Use Adam optimizer (adaptive learning rate)")
print("   - Batch size of 32 works well for most cases")
print("   - Set max epochs high and use early stopping")

print("\n5. EVALUATION")
print("   - Always use a held-out test set for final evaluation")
print("   - Monitor multiple metrics (AUC, F1, not just accuracy)")
print("   - Plot learning curves to diagnose issues")

print("\n6. MODEL SELECTION")
print("   - For small-medium tabular datasets, tree-based models often match NN")
print("   - Neural networks shine with more data and complex patterns")
print("   - Consider ensemble of both approaches")

In [ ]:
# Save the best neural network model
# Get the best performing NN based on final comparison
nn_results = final_df[final_df['Type'].str.contains('Neural')]
best_nn = nn_results.iloc[0]

print(f"Best Neural Network Model: {best_nn['Model']}")
print(f"Test AUC: {best_nn['Test AUC']}")

# Save the dropout model as it's likely one of the best
model_dropout.save(f'{models_path}best_neural_network.keras')
print(f"\nSaved best neural network to: {models_path}best_neural_network.keras")

## 9. Summary

In this notebook, we evaluated and tuned neural networks for student departure prediction.

### Key Results

**Test Set Performance Comparison:**

| Model Type | Best AUC | Notes |
|:-----------|:---------|:------|
| Random Forest | Check above | Robust, interpretable |
| Simple NN | Check above | Fast, few parameters |
| Deep NN | Check above | More complex patterns |
| Dropout NN | Check above | Better regularization |

### Regularization Techniques

| Technique | Purpose | Implementation |
|:----------|:--------|:---------------|
| **Dropout** | Prevents over-reliance on neurons | `Dropout(0.3)` layer |
| **Early Stopping** | Stops when val_loss plateaus | `EarlyStopping(patience=15)` |
| **Learning Rate Decay** | Finer convergence | `ReduceLROnPlateau()` |
| **Class Weights** | Handles imbalance | `class_weight={0: w0, 1: w1}` |

### Architecture Tuning Insights

1. **Depth**: 2-3 hidden layers often sufficient for tabular data
2. **Width**: 16-64 neurons per layer is reasonable
3. **Shape**: Funnel (wide to narrow) works well
4. **More parameters != better performance**

### Neural Networks vs. Tree-Based Models

For our student departure dataset:
- Both approaches achieve competitive performance
- Tree-based models are faster and more interpretable
- Neural networks may improve with more data
- Consider ensemble approaches for best results

### Module 5 Complete!

You have now learned:
- How neural networks work (perceptrons, layers, backpropagation)
- How to build networks with Keras Sequential API
- How to train with epochs, batches, and callbacks
- How to evaluate and tune neural network architectures
- When to use neural networks vs. other approaches

**Congratulations on completing Module 5!**